# Кластеризация. Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn


## Подготовка данных
Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

    id | user_id | venue_id | latitude | longitude | created_at

    ---------+---------+----------+-------------------+-------------------+---------------------

    984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

    984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

    984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

    984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

    ...
    

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

    id,user_id,venue_id,latitude,longitude,created_at

    984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

    984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

    984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

    ...
    
    
С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = 'checkins.dat'

In [3]:
df = pd.read_csv(path, sep='|')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,---------+---------+----------+---------------...,NaN,NaN,NaN,NaN,NaN
1,984301,2041916.0,5222.0,,,2012-04-21 17:39:01
2,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984315,1764391.0,5222.0,,,2012-04-21 17:37:18
4,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43


In [5]:
# Убираем первую строку
df.drop(labels=0, axis='index', inplace=True)

In [6]:
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984301,2041916.0,5222.0,,,2012-04-21 17:39:01
2,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984315,1764391.0,5222.0,,,2012-04-21 17:37:18
4,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
5,984249,2146840.0,5222.0,,,2012-04-21 17:42:58


In [7]:
# Посмотрим на названия колонок
df.columns

Index(['   id    ', ' user_id ', ' venue_id ', '     latitude      ',
       '     longitude     ', '     created_at      '],
      dtype='object')

In [8]:
# Переименуем колонки: уберём лишние пробелы в названиях
df.columns = ['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at']

In [9]:
# Проверка
df.columns

Index(['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'], dtype='object')

In [10]:
# Убираем лишние пробелы в данных
df['latitude'] = df['latitude'].str.strip()
df['longitude'] = df['longitude'].str.strip()

In [11]:
df.latitude.unique()

array(['', '38.8951118', '33.800745', ..., '51.9934549', '26.192028', nan],
      dtype=object)

In [12]:
df.longitude.unique()

array(['', '-77.0363658', '-84.41052', ..., '-0.3600012', '-80.0964326',
       nan], dtype=object)

In [13]:
# Вместо пустых значений везде ставим NaN
df.replace(to_replace='', value=np.nan, inplace=True)

In [14]:
df.longitude.unique()

array([nan, '-77.0363658', '-84.41052', ..., '8.2383918', '-0.3600012',
       '-80.0964326'], dtype=object)

In [15]:
# Преобразуем строки в числа
df['latitude'] = df['latitude'].astype('float')
df['longitude'] = df['longitude'].astype('float')

In [16]:
# Проверка
df.dtypes

id             object
user_id       float64
venue_id      float64
latitude      float64
longitude     float64
created_at     object
dtype: object

In [17]:
# Убираем строки, в которых нет координат
df.dropna(subset=['latitude', 'longitude'], inplace=True)

In [18]:
df.shape[0] == 396634

True

## Обработка данных

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом `MeanShift`, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в [sklearn user guide](http://scikit-learn.org/stable/modules/clustering.html#mean-shift), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте `MeanShift`, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

*Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.*

In [19]:
from sklearn.cluster import MeanShift, estimate_bandwidth

In [20]:
df.dtypes

id             object
user_id       float64
venue_id      float64
latitude      float64
longitude     float64
created_at     object
dtype: object

In [21]:
# Отберём только первые 100'000 строк и из столбцов выберем только широту и долготу 
df_to_clusterize = df.iloc[:100000][['latitude', 'longitude']]
df_to_clusterize

,latitude,longitude
2,38.895112,-77.036366
4,33.800745,-84.410520
8,45.523452,-122.676207
10,40.764462,-111.904565
11,33.448377,-112.074037
...,...,...
233789,33.575000,-117.725556
233790,37.629349,-122.400087
233794,29.762884,-95.383061
233798,32.802955,-96.769923


In [22]:
df_to_clusterize.dtypes

latitude     float64
longitude    float64
dtype: object

In [23]:
%%time
ms = MeanShift(bandwidth=0.1)
ms.fit(df_to_clusterize)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("Количество кластеров: %d" % n_clusters_)

Количество кластеров: 3231
Wall time: 7min 40s


In [24]:
len(labels)

100000

In [25]:
len(cluster_centers)

3231

In [26]:
len(labels_unique)

3231

In [31]:
# Номера полученных кластеров и координаты их центров
cluster_centers_coordinates = pd.DataFrame({'unique_labels':labels_unique,
                                            'cl_center_latitude': cluster_centers.T[0],
                                            'cl_center_longitude': cluster_centers.T[1]})

In [32]:
cluster_centers_coordinates

,unique_labels,cl_center_latitude,cl_center_longitude
0,0,40.717716,-73.991835
1,1,33.449438,-112.002140
2,2,33.446380,-111.901888
3,3,41.878244,-87.629843
4,4,37.688682,-122.409330
...,...,...,...
3226,3226,-36.506376,148.301203
3227,3227,-37.713377,145.148916
3228,3228,-37.822983,145.181190
3229,3229,-41.292494,174.773235


In [35]:
# Создадим датафрейм, в котором появистся столбец с лейблами кластеров
df_clusterized = df_to_clusterize.copy()
df_clusterized['labels'] = labels

In [41]:
df_clusterized.dtypes

latitude     float64
longitude    float64
labels         int64
dtype: object

In [42]:
df_clusterized.head()

,latitude,longitude,labels
2,38.895112,-77.036366,5
4,33.800745,-84.410520,7
8,45.523452,-122.676207,30
10,40.764462,-111.904565,65
11,33.448377,-112.074037,1


In [50]:
# Соединим таблицы для того, чтобы добавить в основную таблицу информацию о центрах кластеров
df_clusterized = df_clusterized.merge(right=cluster_centers_coordinates,
                                      how='left',
                                      left_on='labels',
                                      right_on='unique_labels')
df_clusterized

,latitude,longitude,labels,unique_labels,cl_center_latitude,cl_center_longitude
0,38.895112,-77.036366,5,5,38.886165,-77.048783
1,33.800745,-84.410520,7,7,33.766636,-84.393289
2,45.523452,-122.676207,30,30,45.523483,-122.676280
3,40.764462,-111.904565,65,65,40.759600,-111.896078
4,33.448377,-112.074037,1,1,33.449438,-112.002140
...,...,...,...,...,...,...
99995,33.575000,-117.725556,50,50,33.650896,-117.752074
99996,37.629349,-122.400087,4,4,37.688682,-122.409330
99997,29.762884,-95.383061,25,25,29.762698,-95.382314
99998,32.802955,-96.769923,19,19,32.803021,-96.769897


In [62]:
# Убираем лишний столбец
df_clusterized.drop(labels='unique_labels', axis='columns', inplace=True)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [51]:
clusters = df_clusterized.groupby(labels).aggregate({'labels': 'count'})
clusters

,labels
0,12506
1,4692
2,3994
3,3363
4,3526
...,...
3226,1
3227,1
3228,1
3229,1


In [52]:
clusters.rename(columns={'labels': 'number_of_points'}, inplace=True)

In [53]:
clusters.columns

Index(['number_of_points'], dtype='object')

In [54]:
# Выбираем только те кластеры, в которых более 15 локаций
clusters[clusters.number_of_points>15]

,number_of_points
0,12506
1,4692
2,3994
3,3363
4,3526
...,...
684,19
727,26
884,23
1343,21


In [55]:
# Проверка
df_clusterized[df_clusterized['labels'] == 0]['labels'].count()

12506

In [56]:
df_clusterized[df_clusterized['labels'] == 1350]['labels'].count()

19

In [57]:
# кластеры, в которых более 15 локаций
selected_clusters = clusters[clusters.number_of_points>15].index
selected_clusters

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
             595,  615,  636,  657,  672,  684,  727,  884, 1343, 1350],
           dtype='int64', length=592)

In [174]:
selected_data = df_clusterized[df_clusterized['labels'].isin(selected_clusters)]
# selected_data = df_clusterized.copy()
selected_data

,latitude,longitude,labels,cl_center_latitude,cl_center_longitude
0,38.895112,-77.036366,5,38.886165,-77.048783
1,33.800745,-84.410520,7,33.766636,-84.393289
2,45.523452,-122.676207,30,45.523483,-122.676280
3,40.764462,-111.904565,65,40.759600,-111.896078
4,33.448377,-112.074037,1,33.449438,-112.002140
...,...,...,...,...,...
99995,33.575000,-117.725556,50,33.650896,-117.752074
99996,37.629349,-122.400087,4,37.688682,-122.409330
99997,29.762884,-95.383061,25,29.762698,-95.382314
99998,32.802955,-96.769923,19,32.803021,-96.769897


При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт [mapcustomizer.com](https://www.mapcustomizer.com/) имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

    38.8951118,-77.0363658

    33.800745,-84.41052

    45.5234515,-122.6762071

    ...

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

    33.751277, -118.188740 (Los Angeles)

    25.867736, -80.324116 (Miami)

    51.503016, -0.075479 (London)

    52.378894, 4.885084 (Amsterdam)

    39.366487, 117.036146 (Beijing)

    -33.868457, 151.205134 (Sydney)
    
    
Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

**Проверьте, что широта и долгота указаны в правильном порядке (сначала широта, затем долгота), что они написаны через пробел. Убедитесь, что другие центры кластеров находятся от своих ближайших офисов на большем расстоянии, чем выбранный вами.**

In [175]:
offices_cities = ['Los Angeles', 'Miami', 'London', 'Amsterdam', 'Beijing', 'Sydney']
offices_latitude = [33.751277, 25.867736, 51.503016, 52.378894, 39.366487, -33.868457]
offices_longitude = [-118.188740, -80.324116, -0.075479, 4.885084, 117.036146, 151.205134]

In [176]:
offices_coordinates = pd.DataFrame(data={'latitude': offices_latitude,
                                         'longitude': offices_longitude},
                                   index=offices_cities)
offices_coordinates

,latitude,longitude
Los Angeles,33.751277,-118.188740
Miami,25.867736,-80.324116
London,51.503016,-0.075479
Amsterdam,52.378894,4.885084
Beijing,39.366487,117.036146
Sydney,-33.868457,151.205134


In [177]:
from math import sin, cos, asin, sqrt, radians

def distance_km(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    Код: https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6373 # https://andrew.hedges.name/experiments/haversine/
    return c * r

In [180]:
offices_coordinates.loc['Los Angeles']['longitude']

-118.18874

In [181]:
selected_data.head()

,latitude,longitude,labels,cl_center_latitude,cl_center_longitude
0,38.895112,-77.036366,5,38.886165,-77.048783
1,33.800745,-84.410520,7,33.766636,-84.393289
2,45.523452,-122.676207,30,45.523483,-122.676280
3,40.764462,-111.904565,65,40.759600,-111.896078
4,33.448377,-112.074037,1,33.449438,-112.002140


In [182]:
import warnings
warnings.filterwarnings("ignore")

In [194]:
selected_data.columns

Index(['latitude', 'longitude', 'labels', 'cl_center_latitude',
       'cl_center_longitude', 'distance_to_LA_office',
       'distance_to_Los Angeles_office', 'distance_to_Miami_office',
       'distance_to_London_office', 'distance_to_Amsterdam_office',
       'distance_to_Beijing_office', 'distance_to_Sydney_office'],
      dtype='object')

In [195]:
for city in offices_coordinates.index:
    selected_data[f'distance_to_{city}_office'] = selected_data.apply(lambda x: distance(offices_coordinates.loc[city, 'latitude'],
                                                                                offices_coordinates.loc[city, 'longitude'],
                                                                                x['cl_center_latitude'],
                                                                                x['cl_center_longitude']),
                                                            axis='columns')

In [196]:
selected_data

,latitude,longitude,labels,cl_center_latitude,cl_center_longitude,distance_to_LA_office,distance_to_Los Angeles_office,distance_to_Miami_office,distance_to_London_office,distance_to_Amsterdam_office,distance_to_Beijing_office,distance_to_Sydney_office
0,38.895112,-77.036366,5,38.886165,-77.048783,11603.236888,3699.443462,1480.057954,5905.365434,6192.529320,11199.793000,15714.312134
1,33.800745,-84.410520,7,33.766636,-84.393289,11960.745983,3110.906396,962.098790,6774.624628,7060.016117,11592.853528,14949.902372
2,45.523452,-122.676207,30,45.523483,-122.676280,11052.328550,1364.080332,4337.385739,7911.438598,8039.139178,8861.109853,12345.824989
3,40.764462,-111.904565,65,40.759600,-111.896078,11457.131551,957.532408,3343.202780,7828.764201,8010.082321,9823.252476,12909.889854
4,33.448377,-112.074037,1,33.449438,-112.002140,11980.774538,574.055217,3162.226372,8482.837392,8686.759951,10477.960671,12557.821517
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,33.575000,-117.725556,50,33.650896,-117.752074,11968.083750,41.921921,3695.004605,8772.875531,8955.238024,10135.607333,12095.909081
99996,37.629349,-122.400087,4,37.688682,-122.409330,11692.469624,580.436739,4151.869087,8629.572642,8782.342926,9520.948879,11947.724493
99997,29.762884,-95.383061,25,29.762698,-95.382314,12194.361286,2197.297809,1542.047071,7809.727942,8077.328013,11632.017335,13822.182865
99998,32.802955,-96.769923,19,32.803021,-96.769897,12020.793670,1990.980267,1768.568741,7643.175673,7898.123444,11270.099707,13826.729729


In [197]:
selected_data[selected_data.columns[5:]].min()

distance_to_LA_office             8733.977235
distance_to_Los Angeles_office       7.485771
distance_to_Miami_office             2.509262
distance_to_London_office            3.465919
distance_to_Amsterdam_office         0.822376
distance_to_Beijing_office          31.318788
distance_to_Sydney_office            0.871176
dtype: float64

In [198]:
min_distance = min(selected_data[selected_data.columns[5:]].min())
min_distance

0.8223761325638167

In [201]:
selected_data[selected_data['distance_to_Amsterdam_office'] == min_distance]

,latitude,longitude,labels,cl_center_latitude,cl_center_longitude,distance_to_LA_office,distance_to_Los Angeles_office,distance_to_Miami_office,distance_to_London_office,distance_to_Amsterdam_office,distance_to_Beijing_office,distance_to_Sydney_office
1498,52.373056,4.892222,370,52.372964,4.892317,10397.900723,8967.518675,7448.060445,354.059289,0.822376,7904.926794,16648.319136
2266,52.373056,4.892222,370,52.372964,4.892317,10397.900723,8967.518675,7448.060445,354.059289,0.822376,7904.926794,16648.319136
6216,52.373056,4.892222,370,52.372964,4.892317,10397.900723,8967.518675,7448.060445,354.059289,0.822376,7904.926794,16648.319136
9459,52.373056,4.892222,370,52.372964,4.892317,10397.900723,8967.518675,7448.060445,354.059289,0.822376,7904.926794,16648.319136
11670,52.373056,4.892222,370,52.372964,4.892317,10397.900723,8967.518675,7448.060445,354.059289,0.822376,7904.926794,16648.319136
15971,52.373056,4.892222,370,52.372964,4.892317,10397.900723,8967.518675,7448.060445,354.059289,0.822376,7904.926794,16648.319136
30924,52.373056,4.892222,370,52.372964,4.892317,10397.900723,8967.518675,7448.060445,354.059289,0.822376,7904.926794,16648.319136
39344,52.373056,4.892222,370,52.372964,4.892317,10397.900723,8967.518675,7448.060445,354.059289,0.822376,7904.926794,16648.319136
44752,52.373056,4.892222,370,52.372964,4.892317,10397.900723,8967.518675,7448.060445,354.059289,0.822376,7904.926794,16648.319136
45309,52.373056,4.892222,370,52.372964,4.892317,10397.900723,8967.518675,7448.060445,354.059289,0.822376,7904.926794,16648.319136
